In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
df = pd.read_csv("Train_TEC_mar17.csv")
df.head()

,Date,Month,Year,Time,LT,TEC
0,1,3,2017,0.016389,5.311333,2.067562
1,1,3,2017,0.049722,5.344670,2.082707
2,1,3,2017,0.083056,5.378006,2.004272
3,1,3,2017,0.116389,5.411343,1.736569
4,1,3,2017,0.149722,5.444679,1.758503


In [3]:
df.size

86400

In [4]:
df.drop(["Date","Month","Year","Time"], inplace = True, axis = 1)
df.head()

,LT,TEC
0,5.311333,2.067562
1,5.344670,2.082707
2,5.378006,2.004272
3,5.411343,1.736569
4,5.444679,1.758503


In [5]:
train = df[:-1440]
test = df[-1440:]
train.to_csv("train.csv")
test.to_csv("test.csv")

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train_tec = train.TEC.values
train_tec = train_tec.reshape((-1,1))
tec_train = train_tec
# train = train.values.reshape((-1,1))
# tec_train = train.TEC

In [7]:
test_tec = test.TEC.values
test_tec = test_tec.reshape((-1,1))
tec_test = test_tec

In [8]:
# tec_vals = df['TEC'].values
# tec_vals = tec_vals.reshape((-1,1))

# tec_train = tec_vals[:-2880]
# tec_test = tec_vals[-2880:]

lt_train = train['LT']
lt_test = test['LT']

print(len(tec_train))
print(len(tec_test))

12960
1440


In [9]:
tec_train.size

12960

In [10]:
tec_test.size

1440

In [11]:
look_back = 15

train_generator = TimeseriesGenerator(tec_train, tec_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(tec_test, tec_test, length=look_back, batch_size=1)

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
648/648 [==============================] - 16s 25ms/step - loss: 1.7283
Epoch 2/25
648/648 [==============================] - 17s 26ms/step - loss: 0.5260
Epoch 3/25
648/648 [==============================] - 16s 25ms/step - loss: 0.3528
Epoch 4/25
648/648 [==============================] - 16s 24ms/step - loss: 0.2586
Epoch 5/25
648/648 [==============================] - 16s 24ms/step - loss: 0.1794
Epoch 6/25
648/648 [==============================] - 15s 23ms/step - loss: 0.1136
Epoch 7/25
648/648 [==============================] - 15s 23ms/step - loss: 0.1138
Epoch 8/25
648/648 [==============================] - 16s 24ms/step - loss: 0.1135
Epoch 9/25
648/648 [==============================] - 17s 26ms/step - loss: 0.0818
Epoch 10/25
648/648 [==============================] - 16s 25ms/step - loss: 0.0972
Epoch 11/25
648/648 [==============================] - 16s 24ms/step - loss: 0.0893
Epoch 12/25
648/648 [==============================] - 16s 25ms/step - loss: 0.0889
E

prediction = model.predict_generator(test_generator)

tec_train = tec_train.reshape((-1))
tec_test = tec_test.reshape((-1))
prediction = prediction.reshape((-1))

plt.plot(tec_test, label='Actual')
plt.plot(prediction, label='Predicted')
plt.title("LSTM's_Prediction")
plt.xlabel('Observation')
plt.ylabel('TEC')
plt.legend()
plt.show()

In [ ]:
res

In [124]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(tec_test, prediction))

rms

ValueError: ignored

In [15]:
res = pd.DataFrame(tec_test[:1425], prediction)

In [17]:
res.head(30)

,0
3.758951,2.192221
3.666481,2.208907
4.079441,2.572185
4.627974,2.812540
5.032678,2.892568
4.981465,3.197028
4.781513,3.051936
5.244559,3.228546
5.422810,3.124095
5.618537,3.164987
